In [ ]:
#target2 x
!tar -xvf /content/drive/MyDrive/cctv민원/train_target1.tar

138.종합_민원_이미지_AI데이터/01.데이터/1.Training/라벨링데이터/TL1.zip.part0


In [ ]:
!7z x /content/138.종합_민원_이미지_AI데이터/01.데이터/1.Training/라벨링데이터/TL5.zip.part0


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/138.종합_민원 . /1.Training/라벨링데이터/                                                                      1 file, 176042112 bytes (168 MiB)

Extracting archive: /content/138.종합_민원_이미지_AI데이터/01.데이터/1.Training/라벨링데이터/TL5.zip.part0
  1% 4096 Open               81% 172032 Open                --
Path = /content/138.종합_민원_이미지_AI데이터/01.데이터/1.Training/라벨링데이터/TL5.zip.part0
Type = zip
Physical Size = 176042112
64-bit = +

  0%      0% 746          0% 1129 - 쓰레기/도로변비포장쓰레기(밤)/ . 2_6927-0-2000.json

In [ ]:
!unzip -O CP949 -o -qq /content/drive/MyDrive/cctv민원/yolo_training.zip

In [ ]:
import os
import shutil

def move_files(source_folder, destination_folder):
    # 소스 폴더의 모든 파일을 가져옵니다.
    files = os.listdir(source_folder)

    # 목적지 폴더가 존재하지 않으면 생성합니다.
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # 각 파일을 목적지 폴더로 이동합니다.
    for file in files:
        source_file = os.path.join(source_folder, file)
        destination_file = os.path.join(destination_folder, file)

        # 파일이 존재하는지 확인하고, 파일인 경우에만 이동합니다.
        if os.path.isfile(source_file):
            shutil.move(source_file, destination_file)
        else:
            print(f'Skipped: {source_file} (not a file)')

# 예시 사용법
source_folder = '/content/쓰레기/도로변쓰레기봉투더미(밤)'
destination_folder = '/content/쓰레기'
move_files(source_folder, destination_folder)

In [ ]:
import os
import glob

def clean_json_files(image_folder, json_folder):
    # 이미지 파일 이름만 추출 (확장자 제거)
    image_files = [os.path.splitext(f)[0] for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]

    # JSON 파일 리스트
    json_files = [os.path.splitext(f)[0] for f in os.listdir(json_folder) if f.lower().endswith('.json')]

    # 이미지와 매칭되는 JSON 파일 이름 리스트
    matching_json_files = set(image_files) & set(json_files)

    # JSON 폴더의 모든 파일 중 매칭되지 않는 파일 삭제
    for json_file in json_files:
        if json_file not in matching_json_files:
            os.remove(os.path.join(json_folder, json_file + '.json'))

    print("작업이 완료되었습니다.")

# 사용 예시
image_folder = '/content/content/yolo_training/images/train'
json_folder = '/content/쓰레기'
clean_json_files(image_folder, json_folder)

작업이 완료되었습니다.


In [ ]:
folder_path = '/content/content/yolo_training/labels/train'

# 이미지 파일의 확장자
image_extensions = ['.jpg', 'jpeg', 'json','txt']

# 폴더 안의 이미지 파일 개수를 세는 함수를 정의합니다.
def count_images_in_folder(folder_path, extensions):
    import os
    count = 0
    # 폴더 안의 모든 파일을 순회합니다.
    for file in os.listdir(folder_path):
        if any(file.lower().endswith(ext) for ext in extensions):
            count += 1
    return count

# 이미지 파일의 개수를 세고 출력합니다.
image_count = count_images_in_folder(folder_path, image_extensions)
print(f'폴더 내 이미지 파일 개수: {image_count}')

폴더 내 이미지 파일 개수: 105000


In [ ]:
import json
import os

# 디렉토리 설정
image_dir = '/content/content/yolo_training/images/train'
label_dir = '/content/content/yolo_training/labels/train'
json_dir = '/content/쓰레기'

# 이미지 크기 설정 (모든 이미지가 동일한 크기인 경우)
img_width = 416
img_height = 416

# 카테고리 그룹 매핑
category_mapping = {}
# 그룹 1
for i in range(1, 9):
    category_mapping[i] = 0
# 그룹 2
for i in range(9, 13):
    category_mapping[i] = 1
# 그룹 3
for i in range(13, 21):
    category_mapping[i] = 2
# 그룹 4
for i in range(21, 33):
    category_mapping[i] = 3
# 그룹 5
for i in range(33, 37):
    category_mapping[i] = 4

# JSON 파일 목록 가져오기
json_files = [f for f in os.listdir(json_dir) if f.endswith('.json')]

for json_file in json_files:
    json_file_path = os.path.join(json_dir, json_file)

    # JSON 파일 읽기
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    annotations = data["annotations"]

    for anno_key, anno_value in annotations.items():
        img_file_name = anno_value["atchFileName"]
        label_file_name = img_file_name.replace('.jpg', '.txt')

        boxes = anno_value["Box"]

        # 라벨 파일 작성
        label_file_path = os.path.join(label_dir, label_file_name)
        with open(label_file_path, 'w') as lf:
            for box in boxes:
                category_id = box["category_id"]

                # category_id를 새로운 클래스로 매핑
                if category_id in category_mapping:
                    category_id = category_mapping[category_id]
                else:
                    continue  # 매핑되지 않은 category_id는 무시

                x = box["x"]
                y = box["y"]
                w = box["w"]
                h = box["h"]

                # YOLO 형식으로 변환
                x_center = (x + w / 2) / img_width
                y_center = (y + h / 2) / img_height
                width = w / img_width
                height = h / img_height

                # 라벨 파일에 기록
                lf.write(f"{category_id} {x_center} {y_center} {width} {height}\n")

print("변환 완료!")

변환 완료!


In [ ]:
import os

# 디렉토리 설정
label_dir = '/content/content/yolo_training/labels/train'

# 카테고리 그룹 매핑
category_mapping = {}
# 그룹 1
for i in range(1, 9):
    category_mapping[i] = 0
# 그룹 2
for i in range(9, 13):
    category_mapping[i] = 1
# 그룹 3
for i in range(13, 21):
    category_mapping[i] = 2
# 그룹 4
for i in range(21, 33):
    category_mapping[i] = 3
# 그룹 5
for i in range(33, 37):
    category_mapping[i] = 4

# 라벨 파일 목록 가져오기
label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

for label_file in label_files:
    label_file_path = os.path.join(label_dir, label_file)

    with open(label_file_path, 'r') as lf:
        lines = lf.readlines()  # 모든 행을 읽어들임

    new_lines = []
    for line in lines:
        parts = line.strip().split()
        category_id = int(parts[0])

        # category_id를 새로운 클래스로 매핑
        if category_id in category_mapping:
            new_category_id = category_mapping[category_id]

            # 변경된 category_id로 새로운 라인을 생성
            new_line = f"{new_category_id} " + " ".join(parts[1:]) + "\n"
            new_lines.append(new_line)
        else:
            continue  # 매핑되지 않은 category_id는 무시

    # 기존 파일에 새로운 내용으로 덮어쓰기
    with open(label_file_path, 'w') as lf:
        lf.writelines(new_lines)

print("모든 라벨 파일이 변환되었습니다.")

모든 라벨 파일이 변환되었습니다.


In [ ]:
import os

def get_unique_prefixes(directory):
    # 지정된 디렉토리의 파일 목록을 가져옵니다.
    files = os.listdir(directory)

    # 파일 목록에서 첫 두 글자를 추출하여 집합(set)으로 만듭니다.
    prefixes = set()
    for file in files:
        if len(file) >= 2:  # 파일 이름이 최소 2글자 이상인 경우에만 처리
            prefixes.add(file[:2])

    return prefixes

def get_non_matching_prefixes(dir1, dir2):
    # 두 디렉토리의 파일 목록에서 첫 두 글자를 추출합니다.
    prefixes1 = get_unique_prefixes(dir1)
    prefixes2 = get_unique_prefixes(dir2)

    # 두 디렉토리에서 매칭되지 않는 접두사를 찾습니다.
    non_matching_prefixes = prefixes1.symmetric_difference(prefixes2)

    return non_matching_prefixes

# 예시 디렉토리 경로
directory1 = '/content/yolo_training/labels/train'  # 첫 번째 폴더 경로로 변경하세요.
directory2 = '/content/yolo_training/images/train'  # 두 번째 폴더 경로로 변경하세요.

non_matching_prefixes = get_non_matching_prefixes(directory1, directory2)

print("Non-matching prefixes:", non_matching_prefixes)

Non-matching prefixes: {'3_', '4_', '8_', '6_', '7_', '1_', '5_', '2_'}


In [ ]:
%cd /content/drive/MyDrive/cctv민원

/content/drive/MyDrive/cctv민원


In [ ]:
!zip -qq -r yolo_training.zip /content/content/yolo_training

In [ ]:
import os

# 디렉토리 설정
base_dir = '/content/content/yolo_training'
image_dir = os.path.join(base_dir, 'images')
label_dir = os.path.join(base_dir, 'labels')

# 데이터셋 경로 설정
train_images = os.path.join(image_dir, 'train')

# 클래스 이름 설정 (카테고리 그룹에 맞게 이름을 지정)
classes = [
    '쓰레기 (1-8)',
    '현수막 (9-12)',
    '공사현장 (13-20)',
    '보행방해물 (21-32)',
    '불법주정차 (33-36)'
]

# data.yaml 파일 경로
data_yaml_path = os.path.join(base_dir, 'data.yaml')

# data.yaml 파일 작성
with open(data_yaml_path, 'w') as f:
    f.write(f'train: {train_images}\n')
    f.write(f'nc: {len(classes)}\n')
    f.write('names: [\n')
    for i, cls in enumerate(classes):
        f.write(f'  "{cls}"')
        if i < len(classes) - 1:
            f.write(',\n')
        else:
            f.write('\n')
    f.write(']\n')

print(f"{data_yaml_path} 파일이 생성되었습니다.")

/content/content/yolo_training/data.yaml 파일이 생성되었습니다.
